<a href="https://colab.research.google.com/github/mvgarcia/Reto-Rappi/blob/main/Simple_Model_YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Primer Modelo: **Roboflow**

#Instalar las dependencias

Instalamos las dependencias y seleccionamos la GPU

In [ ]:
# Clonamos el repositorio de YOLOv5
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

In [ ]:
# Instalamos las dependencias necesarias 
!pip install -qr requirements.txt  
import torch

from IPython.display import Image, clear_output  # Para visualizar las imágenes
from utils.google_utils import gdrive_download   # Para descargar los modelos y datasets

In [ ]:
# Export code snippet and paste here
%cd /content
!curl -L "https://app.roboflow.com/ds/3B1r90N6De?key=YtRIOS4p2Q" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
# Este es el archivo YAML de Roboflow necesario para el funcionamiento de los datos. Aquí se reciben el ID de las 6 clases
%cat data.yaml

train: ../train/images
val: ../valid/images

nc: 6
names: ['0', '1', '2', '3', '4', '5']

# Definición del modelo y la arquitectura

Utilizamos el script YAML para definir los parámetros de nuestro modelo

In [ ]:
# Definimos el número de clases de YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
# Corremos la configuración de un modelo base
%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
# Customizamos los archivos de IPython para poder escribir variables sobre ellos
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

Sobreescribimos nuestras clases sobre el modelo base llamado

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# Parámetros
nc: {num_classes}  # Número de clases
depth_multiple: 0.33  # Modelo depth 
width_multiple: 0.50  # Layer channel 

# Anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 
backbone:
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

# Detector de imágenes YOLOv5 personalizado

### Luego, iniciamos los metaparámetros del entrenamiento


A esta función de los metaparámetros le podemos agregar un valor personalizado
- **img:** Tamaño de la imagen
- **batch:** Tamaño del _batch_
- **epochs:** Definimos el número de épocas. 
- **data:** La dirección de nuestro archivo YAML
- **cfg:** Especificamos la configuración de nuestro modelo
- **weights:** Podemos especificar los pesos por transfer learning si los tenemos. En este caso, entrenamos un algoritmo desde 0.
- **name:** Nombres de los resultados.
- **nosave:** Sólo guardar el checkpoint final.
- **cache:** Imágenes caché para un entrenamiento más rápido.

In [ ]:
# Entrenamos YOLOv5 con 200 épocas y 32 de batch size
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 32 --epochs 200 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/content/yolov5
github: ⚠️ WARNING: code is out of date by 514 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 v4.0-126-g886f1c0 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Namespace(adam=False, batch_size=32, bucket='', cache_images=True, cfg='./models/custom_yolov5s.yaml', data='../data.yaml', device='', entity=None, epochs=200, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='yolov5s_results', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov5s_results4', single_cls=False, sync_bn=False, total_batch_size=32, weights='', workers=8, world_size=1)
wandb: Install Weights & Biases for YOLOv5 logging with 'pip install wandb' (recommended)
Start Tensorboard with "tensorboard --

# Evaluamos nuestro detector de YOLOv5

Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov5s_results`. (If given no name, it defaults to `results.txt`.) The results file is plotted as a png after training completes.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
# Iniciamos nuestra pantalla de tensor
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# Podemos ver los valores de nuestra data verdadera
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/test_batch0_labels.jpg', width=900)

In [ ]:
# Y los de la data predecida
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)